In [1]:
import json
import numpy as np

# File paths
meta_file = r"C:\Users\zadec\Desktop\Cornell Tech\Deep Learning\Final Project\real\meta_Beauty_and_Personal_Care.jsonl\meta_Beauty_and_Personal_Care.jsonl"
beauty_file = r"C:\Users\zadec\Desktop\Cornell Tech\Deep Learning\Final Project\real\Beauty_and_Personal_Care.jsonl\Beauty_and_Personal_Care.jsonl"

# Load parent_asin counts from the beauty file
parent_asin_counts = {}
with open(beauty_file, 'r') as fp:
    for line in fp:
        entry = json.loads(line)
        parent_asin = entry.get('parent_asin')
        if parent_asin:
            if parent_asin in parent_asin_counts:
                parent_asin_counts[parent_asin] += 1
            else:
                parent_asin_counts[parent_asin] = 1

# Calculate matching parent_asin counts for meta beauty entries
matching_counts = []
with open(meta_file, 'r') as fp:
    for line in fp:
        entry = json.loads(line)
        parent_asin = entry.get('parent_asin')
        if parent_asin:
            matching_counts.append(parent_asin_counts.get(parent_asin, 0))

# Calculate average and standard deviation
average = np.mean(matching_counts)
std_dev = np.std(matching_counts)

print(f"Average number of matching parent_asins: {average}")
print(f"Standard deviation: {std_dev}")

Average number of matching parent_asins: 23.239444696058175
Standard deviation: 172.20258567860085


In [2]:
import json
import pandas as pd
from spacy.matcher import PhraseMatcher
import spacy
import re
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import os
import random

# Load spaCy model
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "tagger"])

# Define ingredients and compile regex pattern
ingredients = [
    "Allantoin", "Alcohol Denat", "Almond Oil", "Aloe Vera", "Alpha Hydroxy Acid", "Amala Oil", "Amino Acids",
    "Amoxicillin", "Antioxidants", "Apple Cider Vinegar", "Apricot Kernel Oil", "Arbutin", "Argan Oil",
    "Argireline", "Ascorbyl Glucoside", "Astaxanthin", "Avocado Oil", "Azelaic Acid", "Azulene", "Baobab",
    "Baking Soda", "Bakuchiol", "Bentonite Clay", "Benzoyl Peroxide", "Benzyl Alcohol", "Beta Glucan",
    "Bhringraj Oil", "Biotin", "Bio Oil", "Black Cumin Seed Oil", "Borage Seed Oil", "Butylene Glycol", "CBD Oil",
    "CBD", "Caffeine", "Calamine Lotion", "Camellia Extract", "Capric Triglyceride", "Caprylyl Glycol", "Carbomer",
    "Caviar Extract", "Carrier Oils", "Carrot", "Castor Oil", "Cephalexin", "Ceramides", "Cetearyl Alcohol",
    "Chamomile", "Charcoal", "Chebula", "Chia Seed Oil", "Citric Acid", "Cocamidopropyl-Betaine", "Cocoa Butter",
    "Coconut Oil", "Collagen", "Colloidal Oatmeal", "Cone Snail Venom", "Copper Peptides", "CoQ10",
    "Cyclopentasiloxane", "Cypress Oil", "Desitin", "Dihydroxyacetone", "Dimethicone", "Doxycycline", "Emollients",
    "Emu Oil", "Epsom Salt", "Eucalyptus Oil", "Evening Primrose Oil", "Ferulic Acid", "Fermented Oils",
    "Frangipani", "Gluconolactone", "Glycerin", "Glycolic Acid", "Goat's Milk", "Goji Berry", "Gold",
    "Grapeseed Oil", "Green Tea", "Hemp Oil", "Homosalate", "Honey", "Humectants", "Hyaluronic Acid",
    "Hydrocortisone", "Hydrogen Peroxide", "Hydroquinone", "Isododecane", "Isoparaffin", "Isopropyl Myristate",
    "Jojoba", "Kaolin", "Karanja Oil", "Kigelia Africana", "Kojic Acid", "Kukui Nut Oil", "Lactic Acid",
    "Lactobionic Acid", "Lanolin", "Lavender Oil", "Lemon Juice", "Licorice Extract", "Lysine", "Madecassoside",
    "Magnesium", "Magnesium Aluminum Silicate", "Malic Acid", "Mandelic Acid", "Manuka Honey",
    "Marshmallow Root Extract", "Marula Oil", "Meadowfoam", "Methylparaben", "Mineral Oil", "Moringa Oil",
    "Murumuru Butter", "Muslin", "Neem Oil", "Niacinamide", "Nizoral", "Oat", "Octinoxate", "Octisalate",
    "Octocrylene", "Olive Oil", "Omega Fatty Acids", "Oxybenzone", "Panthenol", "Parabens", "Peppermint Oil",
    "Petroleum Jelly", "PHA", "Phenoxyethanol", "Phytic Acid", "Phytosphingosine", "Placenta", "Plum Oil",
    "Polyglutamic Acid", "Polypeptides", "Pomegranates", "Prickly Pear Oil", "Probioitics", "Progeline",
    "Propanediol", "Propolis", "Propylene Glycol", "Propylparabens", "Purslane Extract", "Pycnogenol",
    "Quercetin", "Reishi Mushrooms", "Resveratrol", "Retin-A", "Retinaldehyde", "Retinol", "Retinyl Palmitate",
    "Rosehip Oil", "Rosemary", "Royal Jelly", "Safflower Oil", "Salicylic Acid", "Sea Buckthorn Oil", "Sea Salt",
    "Seaweed", "Sea Moss", "Shea Butter", "Silver", "Snail Mucin", "Sodium Ascorbyl Phosphate", "Sodium Deoxycholate",
    "Sodium Hyaluronate", "Sodium Hydroxide", "Sodium Lauroyl Lactylate", "Sodium Lauryl Sulfate", "Sodium Palmate",
    "Sodium PCA", "Sodium Tallowate", "Soybean Oil", "Spironolactone", "Stearic Acid", "Stearyl Alcohol",
    "Squalane", "Stem Cells", "Succinic Acid", "Sulfates", "Sulfur", "Sunflower Oil", "Synthetic Beeswax", "Talc",
    "Tamanu Oil", "Tea Tree Oil", "Tepezcohuite", "Tranexamic Acid", "Tretinoin", "Triethanolamine", "Turmeric",
    "Undecylenic Acid", "Urea 40", "Vegetable Glycerin", "Vitamin A", "Vitamin B3", "Vitamin C", "Vitamin D",
    "Vitamin E", "Vitamin F", "Vitamin K", "Volcanic Ash", "Willow Bark Extract", "Xanthan Gum", "Zinc"
]
ingredients_pattern = "|".join([re.escape(ingredient.lower()) for ingredient in ingredients])
ingredients_regex = re.compile(ingredients_pattern)

# Define skin features
skin_features = {
    "Normal Skin": ["normal", "balanced", "healthy", "clear", "untroubled", "even"],
    "Oily Skin": ["oily", "greasy", "shiny", "excess sebum", "slick", "glossy"],
    "Combination Skin": ["combination", "mixed", "dry and oily", "dual type", "patchy oily"],
    "Sensitive Skin": ["sensitive", "irritation", "reactive", "allergic", "fragile", "delicate", "easily irritated", "prone to redness"],
    "Acne": ["acne", "pimple", "blemish", "breakout", "zits", "cystic acne", "spots", "acne-prone", "comedones"],
    "Hydration": ["hydrating", "moisture", "moisturizing", "replenish", "quenched", "hydrated", "plumping", "water retention", "moist"],
    "Pores": ["pores", "enlarged pores", "pore size", "clogged pores", "minimize pores", "visible pores", "pore congestion"],
    "Fine Lines and Wrinkles": ["wrinkles", "fine lines", "aging", "anti-aging", "crow's feet", "expression lines", "laugh lines", "age lines", "crinkles"],
    "Sagging": ["sagging", "loose", "loss of firmness", "drooping", "lack of elasticity", "laxity", "lifting", "skin slackening", "gravity-prone"],
    "Dark Spots": ["dark spots", "hyperpigmentation", "discoloration", "sun spots", "age spots", "melasma", "uneven tone", "brown patches", "pigmented spots"],
    "Redness": ["redness", "red patches", "inflammation", "rosacea", "flushed", "red blotches", "irritated skin", "blushing", "hyperemia"],
    "Uneven Texture": ["uneven texture", "rough", "bumpy", "textured", "dull", "grainy", "coarse", "patchy", "irregular texture"],
    "Dark Circles": ["dark circles", "under-eye", "eye bags", "puffy eyes", "tired eyes", "under-eye discoloration", "shadow", "hollows", "dark under-eyes"]
}

def extract_unique_ingredients_regex(description, ingredients_regex):
    matches = ingredients_regex.findall(description.lower())
    return sorted(set(matches))

def extract_unique_skin_features(text, nlp, skin_features):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    for feature, related_phrases in skin_features.items():
        patterns = [nlp.make_doc(phrase) for phrase in [feature] + related_phrases]
        matcher.add(feature, patterns)
    doc = nlp(text)
    matches = matcher(doc)
    unique_features = {nlp.vocab.strings[match_id].lower() for match_id, _, _ in matches}
    return sorted(unique_features)

# Load data
print("Loading product data...")
data = []
with open(r'C:\Users\zadec\Desktop\Cornell Tech\Deep Learning\Final Project\real\meta_Beauty_and_Personal_Care.jsonl\meta_Beauty_and_Personal_Care.jsonl', 'r') as f:
    lines = f.readlines()
    sample_size = int(len(lines) * 0.1)
    sample_lines = random.sample(lines, sample_size)
    for line in tqdm(sample_lines, desc="Reading JSON lines"):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping line due to error: {e}")

# Create DataFrame
print("Creating products DataFrame...")
products = pd.DataFrame.from_records(data)
products['combined_description'] = products['features'].astype(str) + " " + products['description'].astype(str)
products['extracted_ingredients'] = products['combined_description'].apply(
    lambda desc: extract_unique_ingredients_regex(desc, ingredients_regex)
)
products['skin_features'] = products['combined_description'].apply(
    lambda desc: extract_unique_skin_features(desc, nlp, skin_features)
)

# Calculate averages
average_reviews_per_product = len(products) / products['parent_asin'].nunique()
average_ingredients_per_description = products['extracted_ingredients'].apply(len).mean()
average_attributes_per_review = products['skin_features'].apply(len).mean()

# Calculate distributions
ingredient_distribution = products['extracted_ingredients'].explode().value_counts(normalize=True)
attribute_distribution = products['skin_features'].explode().value_counts(normalize=True)

# Print results
print(f"Average Number of Reviews per Product: {average_reviews_per_product:.2f}")
print(f"Average Number of Ingredients per Description: {average_ingredients_per_description:.2f}")
print(f"Average Number of Attributes per Review: {average_attributes_per_review:.2f}")

print("\nIngredient Distribution:")
print(ingredient_distribution)

print("\nAttribute Distribution:")
print(attribute_distribution)

Loading product data...


Reading JSON lines: 100%|██████████| 102891/102891 [00:05<00:00, 20397.35it/s]


Creating products DataFrame...


c:\Users\zadec\anaconda3\envs\DLF\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x=ingredient_distribution.index, y=ingredient_distribution.values)
plt.xticks(rotation=90)
plt.title('Ingredient Distribution')
plt.xlabel('Ingredients')
plt.ylabel('Proportion')
plt.tight_layout()
plt.show()

# Plot attribute distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=attribute_distribution.index, y=attribute_distribution.values)
plt.xticks(rotation=90)
plt.title('Attribute Distribution')
plt.xlabel('Attributes')
plt.ylabel('Proportion')
plt.tight_layout()
plt.show()